# Module Optimization

In [1]:
import os

import dspy
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

# Configure OpenAI
lm = dspy.LM(
    model="openai/gpt-5", 
    temperature=1.0, 
    max_tokens=16000,
    api_key=os.getenv("OPENAI_API_KEY"))
dspy.configure(lm=lm)


## Synthetic dataset for financial news sentiment analysis

In [2]:
# Training set
train = [
    dspy.Example(
        headline="TechCorp reports record-breaking quarterly earnings, exceeding analyst expectations.",
        answer="Up"
    ).with_inputs("headline"),
    dspy.Example(
        headline="TechCorpfaces regulatory fines due to compliance issues in overseas markets.",
        answer="Down"
    ).with_inputs("headline"),
    dspy.Example(
        headline="TechCorp announces new product launch, expected to capture significant market share.",
        answer="Up"
    ).with_inputs("headline"),
    dspy.Example(
        headline="TechCorp's CEO resigns amid internal investigation.",
        answer="Down"
    ).with_inputs("headline"),
    dspy.Example(
        headline="TechCorp stock trading volume steady, market sentiment mixed.",
        answer="Neutral"
    ).with_inputs("headline"),
]

# Development set
dev = [
    dspy.Example(
        headline="TechCorp secures major government contract for AI solutions.",
        answer="Up"
    ).with_inputs("headline"),
    dspy.Example(
        headline="TechCorp reports supply chain disruptions impacting production.",
        answer="Down"
    ).with_inputs("headline"),
    dspy.Example(
        headline="TechCorp maintains stable dividend payout.",
        answer="Neutral"
    ).with_inputs("headline"),
]

## Define DSPy signatures/modules

In [5]:
# Define DSPy signature for stock price movement prediction
class StockPricePrediction(dspy.Signature):
    """
    Predict whether a stock's price will go up, down, or remain neutral based on the news headline.
    """
    headline: str = dspy.InputField(desc="A news headline about a company's stock price.")
    answer: str = dspy.OutputField(desc="The predicted sentiment of the stock price: 'Up', 'Down', or 'Neutral'.")

# Define DSPy module with chain-of-thought
class StockPriceCoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(StockPricePrediction)

    def forward(self, headline: str) -> str:
        return self.prog(headline=headline)

# Define simple evaluation metric
def stock_prediction_metric(example, pred, trace=None):
    return example.answer.lower() == pred.answer.lower()

# Initialize and compile module
config = dict(
    max_bootstrapped_demos = 4,
    max_labeled_demos = 4,
    num_candidate_programs = 8,
    num_threads = 4
)
optimizer = BootstrapFewShotWithRandomSearch(metric=stock_prediction_metric, **config)
compiled_cot = optimizer.compile(StockPriceCoT(), trainset=train, valset=dev)

# Save optimized module
compiled_cot.save("data/stock_price_cot.json")

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 8 candidate sets.
Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:00<00:00, 137.86it/s]

2025/09/26 20:57:04 INFO dspy.evaluate.evaluate: Average Metric: 3 / 3 (100.0%)



New best score: 100.0 for seed -3
Scores so far: [100.0]
Best score so far: 100.0
Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:00<00:00, 164.22it/s]

2025/09/26 20:57:04 INFO dspy.evaluate.evaluate: Average Metric: 3 / 3 (100.0%)



Scores so far: [100.0, 100.0]
Best score so far: 100.0


...


Scores so far: [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Best score so far: 100.0


 60%|██████    | 3/5 [00:00<00:00, 40.47it/s]


Bootstrapped 3 full traces after 3 examples for up to 1 rounds, amounting to 3 attempts.
Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:00<00:00, 166.61it/s]

2025/09/26 20:57:05 INFO dspy.evaluate.evaluate: Average Metric: 3 / 3 (100.0%)



Scores so far: [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Best score so far: 100.0
11 candidate programs found.


In [6]:
# Example usage: predict stock price movement for a new headline
new_headline = "TechCorp announces breakthorugh in renewable energy technology."
prediction = compiled_cot(headline=new_headline)

print(f"Headline: {new_headline}")
print(f"Prediction: {prediction.answer}")

# Evaluate on dev
correct = 0
for example in dev:
    pred = compiled_cot(headline=example.headline)
    if stock_prediction_metric(example, pred):
        correct += 1
accuracy = correct / len(dev)
print(f"Dev accuracy: {accuracy:.2%}")

Headline: TechCorp announces breakthorugh in renewable energy technology.
Prediction: Up
Dev accuracy: 100.00%


# Prompt Optimization

In [19]:
import os

import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import MIPROv2
# TODO: test GEPA with full trace


## Synthetic dataset for portfolio optimization

In [8]:
# Training set
train = [
    dspy.Example(
        headline="TechCorp Q2 earnings soar 25%, beating estimates; AI division grows 40%",
        fundamentals="P/E: 22, Rev Growth: 15%",
        market_conditions="VIX: 18 (moderate)",
        answer="Buy",
        rationale="Positive earnings and strong AI growth outweigh market volatility."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="PharmaCo faces patent expiration; new drug trials delayed.",
        fundamentals="P/E: 30, Rev Growth: -5%",
        market_conditions="VIX: 25 (high)",
        answer="Sell",
        rationale="Patent expiration and negative growth combined with high volatility suggest risk."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="EnergyCorp secures $1B renewable contract; stock steady.",
        fundamentals="P/E: 18, Rev Growth: 10%",
        market_conditions="VIX: 15 (low)",
        answer="Buy",
        rationale="Contract win and solid fundamentals in stable market favor upside."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="RetailerCo reports flat sales amid consumer slowdown",
        fundamentals="P/E: 20, Rev Growth: 0%",
        market_conditions="VIX: 20 (moderate)",
        answer="Hold",
        rationale="Flat sales and stable fundamentals suggest no immediate action."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="AutoCo launches EV model, but supply chain issues persist.",
        fundamentals="P/E: 25, Rev Growth: 8%",
        market_conditions="VIX: 22 (moderate)",
        answer="Hold",
        rationale="EV launch is positive, but supply chain risks and moderate volatility balance out."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="Global Bank fined $500M for compliance failures; CEO resigns.",
        fundamentals="P/E: 15, Rev Growth: 2%",
        market_conditions="VIX: 28 (high)",
        answer="Sell",
        rationale="Significant fine, leadership uncertainty, and high market volatility create a strong sell signal."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="BioGen gets FDA approval for groundbreaking cancer treatment.",
        fundamentals="P/E: 40, Rev Growth: 5%",
        market_conditions="VIX: 16 (low)",
        answer="Buy",
        rationale="Major FDA approval is a significant catalyst that justifies the high P/E in a stable market."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="CloudNet announces partnership with major enterprise software firm.",
        fundamentals="P/E: 35, Rev Growth: 20%",
        market_conditions="VIX: 19 (moderate)",
        answer="Buy",
        rationale="Strategic partnership coupled with strong revenue growth indicates future potential."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="AeroSpace Inc. misses Q3 delivery targets due to labor strike.",
        fundamentals="P/E: 28, Rev Growth: 3%",
        market_conditions="VIX: 21 (moderate)",
        answer="Sell",
        rationale="Missed targets and labor issues suggest operational problems and potential for further downside."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="Utility giant maintains dividend, reports consistent but slow growth.",
        fundamentals="P/E: 19, Rev Growth: 1%",
        market_conditions="VIX: 17 (low)",
        answer="Hold",
        rationale="Company performance is stable and predictable; no strong catalyst for a buy or sell."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="Logistics Co. benefits from e-commerce boom, but fuel costs rise.",
        fundamentals="P/E: 23, Rev Growth: 12%",
        market_conditions="VIX: 23 (moderate)",
        answer="Hold",
        rationale="Positive revenue trend is offset by rising costs and market uncertainty."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="FinTech innovator loses key executive to competitor.",
        fundamentals="P/E: 50, Rev Growth: 30%",
        market_conditions="VIX: 26 (high)",
        answer="Sell",
        rationale="Loss of key talent in a high-growth, high-valuation company amid market volatility is a major red flag."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="Mining Corp discovers new significant mineral deposit.",
        fundamentals="P/E: 16, Rev Growth: 7%",
        market_conditions="VIX: 14 (low)",
        answer="Buy",
        rationale="A major discovery with solid fundamentals and a calm market points to significant upside."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="Media Conglomerate to spin off its streaming division.",
        fundamentals="P/E: 21, Rev Growth: 4%",
        market_conditions="VIX: 20 (moderate)",
        answer="Hold",
        rationale="A corporate restructuring creates uncertainty; best to wait and see how the market values the separate entities."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="ChipMaker warns of lower demand for PCs and smartphones.",
        fundamentals="P/E: 18, Rev Growth: 5%",
        market_conditions="VIX: 24 (high)",
        answer="Sell",
        rationale="A demand warning in a key sector combined with high market volatility suggests a downturn."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="AgriBusiness Inc. sees record crop yields due to favorable weather.",
        fundamentals="P/E: 14, Rev Growth: 9%",
        market_conditions="VIX: 15 (low)",
        answer="Buy",
        rationale="Excellent operational results, a low valuation, and a stable market create a compelling buy case."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="Software Corp announces share buyback program after stock dip.",
        fundamentals="P/E: 26, Rev Growth: 11%",
        market_conditions="VIX: 19 (moderate)",
        answer="Buy",
        rationale="A share buyback signals management confidence in the company's value, suggesting a good entry point."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="Heavy Machinery Co. reports decline in international orders.",
        fundamentals="P/E: 22, Rev Growth: -2%",
        market_conditions="VIX: 22 (moderate)",
        answer="Sell",
        rationale="Declining orders and negative revenue growth point to fundamental weakness."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="Telecom provider expands 5G network to 10 new cities.",
        fundamentals="P/E: 20, Rev Growth: 3%",
        market_conditions="VIX: 18 (moderate)",
        answer="Hold",
        rationale="Network expansion is a positive long-term investment, but slow growth suggests no immediate catalyst."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="Consumer brand faces social media backlash over controversial ad campaign.",
        fundamentals="P/E: 29, Rev Growth: 6%",
        market_conditions="VIX: 27 (high)",
        answer="Sell",
        rationale="Brand damage combined with a high P/E and volatile market presents significant risk."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
]

# Development set
dev = [
    dspy.Example(
        headline="TechCorp's AI division faces regulatory scrutiny in Europe.",
        fundamentals="P/E: 25, Rev Growth: 10%",
        market_conditions="VIX: 22 (elevated)",
        answer="Sell",
        rationale="Regulatory concerns in a major market create uncertainty that outweighs the decent growth."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="PharmaCo announces positive results from Phase 2 trial of new Alzheimer's drug.",
        fundamentals="P/E: 38, Rev Growth: 2%",
        market_conditions="VIX: 16 (low)",
        answer="Buy",
        rationale="Positive trial results for a high-impact drug is a major catalyst, justifying the high P/E."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="EnergyCorp's quarterly profits are stable, in line with expectations.",
        fundamentals="P/E: 17, Rev Growth: 3%",
        market_conditions="VIX: 19 (moderate)",
        answer="Hold",
        rationale="Meeting expectations without any surprises suggests the current price is fair; no need for action."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="RetailerCo acquires smaller online competitor to boost e-commerce presence.",
        fundamentals="P/E: 24, Rev Growth: 5%",
        market_conditions="VIX: 21 (moderate)",
        answer="Buy",
        rationale="Strategic acquisition addresses a key growth area and signals proactive management."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="AutoCo's new EV model receives mixed reviews from early testers.",
        fundamentals="P/E: 27, Rev Growth: 7%",
        market_conditions="VIX: 25 (high)",
        answer="Hold",
        rationale="Mixed reviews create uncertainty about a key product launch, making it prudent to wait for more data."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="Global Bank's dividend yield remains attractive despite slow loan growth.",
        fundamentals="P/E: 14, Rev Growth: 1%",
        market_conditions="VIX: 18 (moderate)",
        answer="Hold",
        rationale="The attractive dividend provides support for the stock, but slow growth limits upside potential."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="BioGen faces new competitor for its top-selling drug.",
        fundamentals="P/E: 33, Rev Growth: 8%",
        market_conditions="VIX: 26 (high)",
        answer="Sell",
        rationale="New competition for a key product threatens future revenue streams, especially with a high valuation and market volatility."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="CloudNet's data center expansion is completed ahead of schedule and under budget.",
        fundamentals="P/E: 40, Rev Growth: 18%",
        market_conditions="VIX: 17 (low)",
        answer="Buy",
        rationale="Excellent operational execution on a key growth project supports the high valuation in a stable market."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="AeroSpace Inc. is downgraded by two major investment banks.",
        fundamentals="P/E: 26, Rev Growth: 4%",
        market_conditions="VIX: 23 (moderate)",
        answer="Sell",
        rationale="Analyst downgrades can trigger institutional selling and indicate underlying problems not yet public."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
    dspy.Example(
        headline="Utility giant plans gradual rate hikes over the next three years.",
        fundamentals="P/E: 20, Rev Growth: 2%",
        market_conditions="VIX: 15 (low)",
        answer="Hold",
        rationale="Planned rate hikes provide future earnings visibility but are not a significant catalyst for immediate upside."
    ).with_inputs("headline", "fundamentals", "market_conditions"),
]

## Define DSPy signature/modules

In [ ]:
class PortfolioRecommendation(dspy.Signature):
    """
    Given a news headline, company fundamentals, and market conditions, recommend whether to Buy, Hold, or Sell a stock.
    """
    headline: str = dspy.InputField(desc="A news headline about a company's stock price.")
    fundamentals: str = dspy.InputField(desc="Fundamental metrics of the company: P/E ratio, revenue growth, etc.")
    market_conditions: str = dspy.InputField(desc="Current market conditions: VIX index, market sentiment, etc.")
    answer: str = dspy.OutputField(desc="The recommended action: 'Buy', 'Hold', or 'Sell'.")
    rationale: str = dspy.OutputField(desc="A reasoning for the recommendation based on the headline, fundamentals, and market conditions.")

# Define DSPy module with chain-of-thought
class PortfolioRecommendationCoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(PortfolioRecommendation)

        # agent call

    def forward(self, headline: str, fundamentals: str, market_conditions: str) -> str:
        return self.prog(
            headline=headline,
            fundamentals=fundamentals,
            market_conditions=market_conditions
        )

# Define simple evaluation metric
def portfolio_recommendation_metric(example, pred, trace=None):
    # check the gold label and the predicted answer are the same
    answer_match = example.answer.lower() == pred.answer.lower()

    # check the predicted answer rationale is relevant to the headline, fundamentals, and market conditions
    from trulens.providers.openai import OpenAI
    provider = OpenAI(model_engine="gpt-4o")
    rationale_match = provider.relevance_with_cot_reasons(
        example.headline,
        pred.rationale
    )[0] > 0.67

    if trace is None: # if we're doing evaluation or optimization
        return (answer_match + rationale_match) / 2.0
    else: # if we're doing bootstrapping, i.e. self-generating good demonstrations of each step
        return answer_match and rationale_match

    
# Initialize and compile module
config = dict(
    max_bootstrapped_demos = 4,
    max_labeled_demos = 4,
    num_threads = 4
)
portfolio_optimizer = MIPROv2(metric=portfolio_recommendation_metric, **config)
compiled_portfolio_cot = portfolio_optimizer.compile(PortfolioRecommendationCoT(), trainset=train, valset=dev)

# Save optimized module
compiled_portfolio_cot.save("data/portfolio_recommendation_cot.json")
    

TypeError: GEPA metric must accept five arguments: (gold, pred, trace, pred_name, pred_trace). See https://dspy.ai/api/optimizers/GEPA for details.

In [13]:
# Example usage: predict portfolio recommendation for a new headline, fundamentals, and market conditions
new_headline = "TechCorp announces breakthrough in renewable energy technology."
new_fundamentals = "P/E: 22, Rev Growth: 5%"
new_market_conditions = "VIX: 27 (high)"

prediction = compiled_portfolio_cot(
    headline=new_headline, 
    fundamentals=new_fundamentals, 
    market_conditions=new_market_conditions
)

print(f"Headline: {new_headline}")
print(f"Fundamentals: {new_fundamentals}")
print(f"Market Conditions: {new_market_conditions}")
print(f"Prediction: {prediction.answer}")
print(f"Rationale: {prediction.rationale}")

# Evaluate on dev
correct = 0
for example in dev:
    pred = compiled_portfolio_cot(
        headline=example.headline,
        fundamentals=example.fundamentals, 
        market_conditions=example.market_conditions)
    if portfolio_recommendation_metric(example, pred):
        correct += 1
accuracy = correct / len(dev)
print(f"Dev accuracy: {accuracy:.2%}")

Headline: TechCorp announces breakthrough in renewable energy technology.
Fundamentals: P/E: 22, Rev Growth: 5%
Market Conditions: VIX: 27 (high)
Prediction: Hold
Rationale: The headline adds promising optionality, but with modest current growth and a valuation that isn’t cheap for 5% revenue growth, the stock lacks downside protection if commercialization is slower or cost curves disappoint. Elevated market volatility increases the risk of headline whipsaws. Wait for concrete proof points—e.g., third-party validation, pilot results with clear cost/margin benefits, IP/patent details, customer MOUs or orders, and financing/partnerships—before upgrading to Buy. Consider trimming into any sharp spike if no substantiating details emerge.
Dev accuracy: 100.00%
